In [46]:
!pip install mlflow seaborn plotly

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import mlflow
import mlflow.sklearn

In [48]:
# Load dataset

# Unsanitized data frame
udf = pd.read_csv("Bank_churn_modelling.csv")

In [49]:
udf.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
udf.shape

(10000, 14)

In [51]:
udf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [52]:
# Drop irrelevant columns
df = udf.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [53]:
# Check if there are any null values
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [54]:
# Check for outliers
# df.skew()
df.select_dtypes(include=['number']).skew()

CreditScore       -0.071607
Age                1.011320
Tenure             0.010991
Balance           -0.141109
NumOfProducts      0.745568
HasCrCard         -0.901812
IsActiveMember    -0.060437
EstimatedSalary    0.002085
Exited             1.471611
dtype: float64

In [55]:
df['Age'].max()

92

In [56]:
maxbelowq99 = df['Age'][df['Age'] < df['Age'].quantile(0.99)].max()
maxbelowq99

71

In [57]:
# Clip age outliers
df[df['Age'] < maxbelowq99].select_dtypes(include=['number']).skew()

CreditScore       -0.070813
Age                0.738466
Tenure             0.010393
Balance           -0.143785
NumOfProducts      0.750154
HasCrCard         -0.900653
IsActiveMember    -0.039134
EstimatedSalary    0.000724
Exited             1.459185
dtype: float64

In [58]:
# Sanitized data frame
df = df[df['Age'] < maxbelowq99]

In [59]:
x = df[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'IsActiveMember']]
y = df['Exited']

In [60]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [61]:
mlflow.set_experiment("customer_churn_prediction")

<Experiment: artifact_location='file:///Users/nisanth/repos/futurense/ml/ml_api_demo/mlruns/788143575997635441', creation_time=1700326409818, experiment_id='788143575997635441', last_update_time=1700326409818, lifecycle_stage='active', name='customer_churn_prediction', tags={}>

In [64]:
mlflow.sklearn.autolog()
with mlflow.start_run():
    transformer = ColumnTransformer([
        ('geo', OneHotEncoder(drop='first'), [1]),
        ('gen', OrdinalEncoder(), [2]),
        ('scaler', StandardScaler(), [0, 3, 4, 5])],
        remainder='passthrough'
    )

    x2 = transformer.fit_transform(x)
    x2 = pd.DataFrame(x2, columns=transformer.get_feature_names_out())
    xtrain, xtest, ytrain, ytest = train_test_split(x2, y, test_size=0.2, random_state=5, stratify=y)
    model = LogisticRegression()
    # Train the model
    model.fit(xtrain, ytrain)

/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [65]:
from sklearn import tree

In [66]:
mlflow.sklearn.autolog()
with mlflow.start_run():
    transformer = ColumnTransformer([
        ('geo', OneHotEncoder(drop='first'), [1]),
        ('gen', OrdinalEncoder(), [2])
    ], remainder='passthrough')

    x2 = transformer.fit_transform(x)
    x2 = pd.DataFrame(x2, columns=transformer.get_feature_names_out())
    xtrain, xtest, ytrain, ytest = train_test_split(x2, y, test_size=0.2, random_state=5, stratify=y)
    model = tree.DecisionTreeClassifier(
        max_depth=30,
        min_samples_leaf=5,
        min_samples_split=20,
        class_weight='balanced'
    )
    # Train the model
    model.fit(xtrain, ytrain)

/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/nisanth/repos/futurense/ml/ml_api_demo/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep